## 2013 Club Championship Stats

In [99]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

In [100]:
stats = pd.read_html('../data/2013ClubChampStats.html')

In [101]:
stats_2013 = stats[0]
stats_2013.head()

,Team,#,Player,Goals,Assists,Ds,TOs
0,Bent,0,Brittany Kaplan,0,3,2,5
1,Bent,1,Lexi Zalk,4,1,4,0
2,Bent,2,Karen Chalif,3,1,0,6
3,Bent,3,Judy Jarvis,3,7,2,11
4,Bent,5,Katherine Cole,1,1,1,2


## 2014 Club Championship Stats

In [102]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [103]:
def get_team_stat_urls(Championship_Page_URL):
    import requests
    from bs4 import BeautifulSoup
    
    r = requests.get(Championship_Page_URL) 
    b = BeautifulSoup(r.text, 'html.parser')

    step1 = [str(item).split('\n') for item in b.find_all('tr', style='border: 1px solid #eee;')]
    step2 = [item[1][13:-9] for item in step1]
    step3 = [item.split('">') for item in step2]
    step4 = [item[1].split(' (') for item in step3]
    
    dict_name = {}
    
    for item in range(16):
        dict_name[step4[item][0]] = step3[item][0] 
    
    return dict_name

In [354]:
teams_2014 = get_team_stat_urls('http://play.usaultimate.org/events/USA-Ultimate-National-Championships/schedule/Women/Club-Women/')

In [355]:
teams_2014

{'Brute Squad': '/events/teams/?EventTeamId=xp%2bejIJBOcfXXkdUBdhyKw7QdaKTsy2msZaYJsN0QFI%3d',
 'Capitals': '/events/teams/?EventTeamId=QMEULDssKTN4OuwrBMNq94efa7qxYCOB39XjXXvDBuY%3d',
 'Fury': '/events/teams/?EventTeamId=CBkbxJkqtPUazzlNKxrMYL476TAvWyT%2b%2bII9vIya7AM%3d',
 'Green Means Go': '/events/teams/?EventTeamId=ejfj3ptI1RiAMMNBDBMR%2bOWMPBwsN0Y86S%2fuCAYPw9U%3d',
 'Heist': '/events/teams/?EventTeamId=h%2bkaG2ZXhVd798X3lGV7PeEc5SUIuvMgQy9BNMokNsM%3d',
 'Molly Brown': '/events/teams/?EventTeamId=cKqY9nt%2bZuiYvmdlJ2MwdG0cK1L9yUN%2bWnowMqR1zKg%3d',
 'Nemesis': '/events/teams/?EventTeamId=mIbs2PBWcyioQl2JAuLt1FjcCr3eIAK7iEJX3hRzIO8%3d',
 'Nightlock': '/events/teams/?EventTeamId=KWGXFcYyimfQtEQ%2fDYoEE7kZ1HzzEC0yN7KIfYlhSWg%3d',
 'Ozone': '/events/teams/?EventTeamId=01oilPzTzWp6xnsGkQ0uJtZl64NEdUOHJ0wQZrvMnNY%3d',
 'Riot': '/events/teams/?EventTeamId=ddLOZCRcEzHqNrt1jat3nz4wNJ%2f7YNKK5OZQonhRuaQ%3d',
 'Scandal': '/events/teams/?EventTeamId=m%2f3%2f0BtHZz5iI3aEgO1fqtpMAlDOwL7dnQ9uEQ

In [356]:
# explore existing columns

r = requests.get('http://play.usaultimate.org' + teams_2014['Traffic']) #any team name will do
team_page = BeautifulSoup(r.text, 'html.parser')

In [357]:
columns_2014 = [str(item)[15:-5] for item in team_page.find('div', id='roster_list').find_all('th', span='col')]

In [358]:
columns_2014

['No.',
 'Name',
 'Position',
 'Year',
 'Height',
 'Goals',
 'Assists',
 'Ds',
 'Turns']

In [359]:
def get_stats_page(team_name, dict_name):
    # get html of team stats page
    r = requests.get('http://play.usaultimate.org' + dict_name[team_name])
    team_page = BeautifulSoup(r.text, 'html.parser')
    return team_page.find('div', id='roster_list')

In [364]:
def player_info(team_name, dict_name):
    
    List1 = [str(item)[4:-5] for item in get_stats_page(team_name, dict_name).find_all('td')]
    
    playerInfo = []
    count = 0
    
    while count < len(List1):
        
        info = {}

        info['No.'] = List1[count]
        info['Name'] = List1[count+1]
#         info['Position'] = List1[count+2]
#         info['Year'] = List1[count+3]
#         info['Height'] = List1[count+4]
        info['Goals'] = int(List1[count+5])
        info['Assists'] = int(List1[count+6])
        info['Ds'] = int(List1[count+7])
        info['Turns'] = int(List1[count+8])
        info['Team'] = team_name
        
        count += 9
        playerInfo.append(info)
    
    return playerInfo

In [365]:
def get_stats(dict_name):
    stats = []

    for team in dict_name:
        stats += player_info(str(team), dict_name)
    
    return stats

In [366]:
import numpy as np
from pandas import DataFrame, Series

stats_14 = DataFrame.from_dict(get_stats(teams_2014))

In [367]:
stats_14.head()

,Assists,Ds,Goals,Name,No.,Team,Turns
0,0,0,0,KARA KELLY,00,Nightlock,0
1,2,0,2,SHARON TSENG,01,Nightlock,0
2,7,0,5,MARISA RAFTER,02,Nightlock,0
3,3,0,3,MICHAELA LEE,03,Nightlock,0
4,5,0,3,MARIKA AUSTIN,04,Nightlock,0


In [369]:
stats_14.dtypes

Assists     int64
Ds          int64
Goals       int64
Name       object
No.        object
Team       object
Turns       int64
dtype: object

In [370]:
stats_14[stats_14.Team == 'Scandal'].sort_values('Goals', ascending=False).head() # sort not working for goals/assists

,Assists,Ds,Goals,Name,No.,Team,Turns
338,4,0,21,SANDRA JORGENSEN,37,Scandal,0
333,5,0,10,ALICIA WHITE,19,Scandal,0
323,4,0,8,SARAH ITOH,04,Scandal,0
325,3,0,7,ASHLEY DALY MORGAN,06,Scandal,0
326,2,0,7,KIMBERLY BEACH,07,Scandal,0


## 2015 Club Championship Stats

In [371]:
teams_2015 = get_team_stat_urls('http://play.usaultimate.org/events/USA-Ultimate-National-Championships-2015/schedule/Women/Club-Women/')

In [372]:
teams_2015

{'BENT': '/events/teams/?EventTeamId=tl1rkVLjdczXVbOWzjG8T6fVjubOyhxGoQqAHV0da4g%3d',
 'Brute Squad': '/events/teams/?EventTeamId=c%2bbyh9S9ANQUYGFX55jj2huoFtowEZg%2fAyga4pprMQ0%3d',
 'Capitals': '/events/teams/?EventTeamId=x4u7Gvp5rOjImw437isTD%2fs%2bQ06953u7JY%2f95O2ZmYY%3d',
 'Fury': '/events/teams/?EventTeamId=32oHgC3wTopGTV3EhDSbH36OX0nyemEg%2bP5KQfBRE1g%3d',
 'Heist': '/events/teams/?EventTeamId=CpQQbzcjJ0UXcqhrXjm3d8zVp0%2fIhQBVr%2fCVAX8yEAY%3d',
 'Iris': '/events/teams/?EventTeamId=Wy3HoIVoyiewP6W0uRSYiIJo8RJY0m3zrLvGpRwtEwI%3d',
 'Molly Brown': '/events/teams/?EventTeamId=RbDtI%2bYRGJIFXwMyY2GXHZNkAP4TG3e5cYp%2bwYagX5s%3d',
 'Nemesis': '/events/teams/?EventTeamId=crhCXpLrJODvUJL6ppk9K1V8i2mXHNLATKv1ZOibPUc%3d',
 'Nightlock': '/events/teams/?EventTeamId=EfdvGasY6mEwfXfzbwlKQKOo3de2IF4mbYrPHnmtd2Q%3d',
 'Ozone': '/events/teams/?EventTeamId=iAImqQuifQ77w%2bC0P2niH56jBIDV%2fpmjWfCpaiDP%2brQ%3d',
 'Phoenix': '/events/teams/?EventTeamId=CB%2br3JHfJtcIvZONww0nNVHSZZMq9jH%2frilvqOvHLy

In [373]:
stats_15 = DataFrame.from_dict(get_stats(teams_2015))

In [379]:
stats_15[stats_15.Team == 'Molly Brown'].sort_values('Goals', ascending=False)

,Assists,Ds,Goals,Name,No.,Team,Turns
230,1,0,10,KRISTINA SNODGRASS,21,Molly Brown,2
218,2,3,7,ASHLEY DALY MORGAN,06,Molly Brown,2
234,2,3,6,TANIA REITZ,27,Molly Brown,1
240,2,2,5,ALICIA WHITE,49,Molly Brown,2
221,3,0,4,LINDSEY CROSS,10,Molly Brown,3
222,7,4,4,DENA SLATTERY,11,Molly Brown,2
239,2,1,4,REBECCA MILLER,47,Molly Brown,3
226,2,1,4,MEGAN COUSINS,16,Molly Brown,3
228,2,1,4,CRYSTAL DAVIS,18,Molly Brown,4
231,2,5,3,LAUREN BOYLE,23,Molly Brown,3
